In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

# Importing dataset
fake_data = pd.read_csv("Fake.csv")
true_data = pd.read_csv("True.csv")

In [2]:
#Load Fake and True datasets
fake_df=pd.DataFrame(fake_data)
true_df=pd.DataFrame(true_data)
#fake_df
#true_df

In [3]:
#Insert a categorical value for Fake (0) or Real (1)
fake_df['Fake'] = '0'
true_df['Fake'] = '1'

In [4]:
#Combine Data
combined=pd.concat([fake_df, true_df])
combined

,title,text,subject,date,Fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
# nltk_tokens = nltk.word_tokenize(combined['text'])
# word_tokenize(combined['text'])
# word_tokenize('hello world')
stopword=stopwords.words('english')
punctuations=list(string.punctuation)
tokenized_list=[]
for each_row in combined['text']:
    word_tokens=word_tokenize(each_row)
    removing_punc=[word for word in word_tokens if word not in punctuations]
    removing_stopwords=[word for word in removing_punc if word not in stopword]
    tokenized_list.append(' '.join(removing_stopwords))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer=CountVectorizer()
# bag_of_words=vectorizer.fit_transform(combined['text'])
# feature_names=vectorizer.get_feature_names()
# bow_df=pd.DataFrame(bag_of_words.toarray(), columns=feature_names)
# bow_df

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(max_features=10000)
vectorizer.fit(tokenized_list)
# need to save this vectorizer to process the inputs
x=vectorizer.transform(tokenized_list)
x

<44898x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 6983972 stored elements in Compressed Sparse Row format>

In [8]:
feature_names=vectorizer.get_feature_names()
len(feature_names)

10000

In [9]:
y=combined['Fake']

In [10]:
from sklearn.model_selection import train_test_split

In [42]:
import timeit
x_train, x_test, y_train, y_test=train_test_split(x.toarray(), y)
nb=GaussianNB()
nb.fit(x_train, y_train)
%time nb.score(x_test, y_test)



CPU times: user 1.1 s, sys: 999 µs, total: 1.1 s
Wall time: 1.1 s


0.9156347438752784

In [43]:
x_train, x_test, y_train, y_test=train_test_split(x.toarray(), y)
Bnb=BernoulliNB()
Bnb.fit(x_train, y_train)
%time Bnb.score(x_test, y_test)


CPU times: user 1.34 s, sys: 157 ms, total: 1.5 s
Wall time: 1.15 s


0.9635634743875279

In [44]:
x_train, x_test, y_train, y_test=train_test_split(x.toarray(), y)
Mnb=MultinomialNB()
Mnb.fit(x_train, y_train)
%time Mnb.score(x_test, y_test)


CPU times: user 549 ms, sys: 97 ms, total: 646 ms
Wall time: 331 ms


0.93728285077951

In [47]:
loaded_vectorizer=pickle.load(open('vectorizer.sav', 'rb'))
loaded_model=pickle.load(open('Bnb_model.sav', 'rb'))
test_sample=loaded_vectorizer.transform([sample_text])
loaded_model.predict(test_sample.toarray())

array(['1'], dtype='<U1')

In [48]:
sample_text=combined.iloc[-1,:]['text']

In [46]:
import pickle
pickle.dump(vectorizer, open('vectorizer.sav', 'wb'))
pickle.dump(gnb, open('Bnb_model.sav', 'wb'))